层和块

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
#nn.Sequential定义了一种特殊的Module，即在PyTorch中表示一个块的类
X = torch.rand(2, 20)
net(X)

tensor([[-0.0354, -0.0347,  0.0409,  0.3840,  0.0430,  0.2867,  0.3209, -0.1176,
         -0.1102,  0.0046],
        [ 0.0210, -0.0994,  0.0049,  0.3062,  0.1077,  0.1662,  0.2721, -0.0070,
         -0.2306,  0.1629]], grad_fn=<AddmmBackward0>)

自定义块
简要总结一下每个块必须提供的基本功能：
  1.将输入数据作为其正向传播函数的参数。
  2.通过正向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收任意维的输入，但是返回一个维度256的输出
  3.计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
  4.存储和访问正向传播计算所需的参数
  5.根据需要初始化模型参数。


In [4]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        #调用‘MLP’的父类‘Block’的构造函数来执行必要的初始化
        #这样，在类实例化时也可以指定其他函数参数，比如模型参数‘params’
        super().__init__()
        self.hidden = nn.Linear(20,256)#隐藏层
        self.out = nn.Linear(256,10)#输出层
    #定义模型的正向传播，即如何根据输入‘X’返回所需的模型输出    
    def forward(self, X):
        #注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[ 0.0158,  0.1295,  0.1967,  0.0287,  0.2187,  0.1926, -0.2091, -0.1338,
          0.1772,  0.0438],
        [ 0.0883,  0.0530,  0.1445, -0.1147,  0.2365,  0.1150, -0.2148, -0.2211,
          0.0321,  0.1609]], grad_fn=<AddmmBackward0>)

顺序块
为了构建我们自己的简化的MySequential，我们只需要定义两个关键函数： 1. 一种将块逐个追加到列表中的函数。 2. 一种正向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            #这里，‘block’是‘Module’子类的一个实例，我们把它保存在‘Module’类的成员变量
            #‘_module’中，‘block’的类型是‘OrderedDict’
            self._modules[block] = block
            #简而言之，_modules的主要优点是，在块的参数初始化过程中，系统知道在_modules字典中查找需要初始化参数的子块。
    def forward(self, X):
        #OrderedDict保证了按照成员添加的顺序遍历他们
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.2214,  0.0283,  0.0833,  0.0479, -0.1829, -0.0717, -0.1398, -0.2181,
         -0.0403,  0.0581],
        [ 0.2890,  0.0511,  0.0968, -0.1045, -0.2460, -0.0266, -0.0380, -0.0905,
         -0.0842,  0.0106]], grad_fn=<AddmmBackward0>)

在正向传播函数中执行代码
Sequential类使模型构造变得简单，允许我们组合新的结构，而不必定义自己的类。然而，并不是所有的架构都是简单的顺序结构。我们需要定义自己的块
有时我们可能希望合并既不是上一层的结果也不是可更新参数的项。我们称之为常数参数（constant parameters）。
在这个FixedHiddenMLP模型中，我们实现了一个隐藏层，其权重（self.rand_weight）在实例化时被随机初始化，之后为常量。这个权重不是一个模型参数，因此它永远不会被反向传播更新。然后，网络将这个固定层的输出通过一个全连接层。

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变。
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及`relu`和`dot`函数。
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数。
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    #在返回输出之前，我们的模型做了一些不寻常的事情。我们运行了一个while循环。
    #我们的重点只是向你展示如何将任意代码集成到神经网络计算的流程中。
net = FixedHiddenMLP()
net(X)

tensor(-0.2225, grad_fn=<SumBackward0>)

参数管理


   访问参数，用于调试、诊断和可视化。

   参数初始化。

   在不同模型组件间共享参数。


In [4]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0289],
        [-0.0130]], grad_fn=<AddmmBackward0>)

参数访问
我们从已有模型中访问参数。当通过Sequential类定义模型时，我们可以通过索引来访问模型的任意层

In [6]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2064,  0.2885, -0.2469,  0.1556, -0.0173,  0.1274, -0.0058, -0.3240]])), ('bias', tensor([0.1748]))])


 延后初始化
 到目前为止，我们忽略了建立网络时需要做的以下这些事情：

   我们定义了网络架构，但没有指定输入维度。

   我们添加层时没有指定前一层的输出维度。

   我们在初始化参数时，甚至没有足够的信息来确定模型应该包含多少参数。

深度学习框架无法判断网络的输入维度是什么。这里的诀窍是框架的延后初始化（defers initialization），即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。
          现在我们在编写代码时无须知道维度是什么就可以设置参数